In [1]:
import os
import copy
import numpy as np
import pandas as pd
from persor import BVHparser
from scipy.spatial.transform import Rotation

In [2]:
def get_joint_coords(skeleton, joint, coords):
    offset = skeleton[joint]["offset"]
    parent_joint = skeleton[joint]["joint"]
    child_joints = skeleton[joint]["children"]

    if parent_joint == None:
        return

    parent_coord = coords[parent_joint]["coord"]

    current_coord = [
        parent_coord[0] + offset[0],
        parent_coord[1] + offset[1],
        parent_coord[2] + offset[2],
    ]
    coords[joint] = {"coord": current_coord, "parent": parent_joint}

    for child in child_joints:
        get_joint_coords(skeleton, child, coords)


def skelton2coords(skeleton):
    coords = {"root": {"coord": skeleton["root"]["offset"], "parent": None}}
    skeleton["root"]

    child_joints = skeleton["root"]["children"]
    for child in child_joints:
        get_joint_coords(skeleton, child, coords)

    get_joint_coords(skeleton, "root", coords)

    return coords


def rotated_offset(skeleton, frame):
    skeleton_copy = copy.deepcopy(skeleton)
    for joint, data in skeleton_copy.items():
        if joint == "root" or joint.startswith("_"):
            continue

        x_rotate = np.deg2rad(frame[f"{joint}_Xrotation"])
        y_rotate = np.deg2rad(frame[f"{joint}_Yrotation"])
        z_rotate = np.deg2rad(frame[f"{joint}_Zrotation"])

        rot = Rotation.from_rotvec(np.array([y_rotate, x_rotate, z_rotate]))
        skeleton_copy[joint]["offset"] = rot.apply(data["offset"])

    return skeleton_copy


def convert2coordDict(df, skeleton):
    df = df.copy()
    data = []
    for i in range(len(df)):
        skeleton_rotated = rotated_offset(skeleton, df.iloc[i])
        coords = skelton2coords(skeleton_rotated)
        data.append(coords)

    return np.array(data)


def coord_dict2coords_array(coords_dict):
    return np.array(
        [[data["coord"] for joint, data in coords.items()] for coords in coords_dict]
    )

def split_by_label_obj(df, label_obj):


In [3]:
# ./no_meaning 以下のファイル名を取得
folder = "./data"
files = [f"{folder}/{f}" for f in os.listdir(folder)]

# 1. 体を曲げる運動
# 2. 両手を上げ下げ
# 3. 体を捻る
# 4. ジャンプ
# 5. 何もしない

## 訓練用データを生成するよ

In [4]:
bending_exercise_bvhp = BVHparser("./data/bvh/bending_exercise.bvh")
bending_exercise_motion_df = bending_exercise_bvhp.get_motion_df()
bending_exercise_motion_df["label"] = 1

hands_updown_bvhp = BVHparser("./data/bvh/hands_updown.bvh")
hands_updown_motion_df = hands_updown_bvhp.get_motion_df()
hands_updown_motion_df["label"] = 2

twist_your_body_bvhp = BVHparser("./data/bvh/twist_your_body.bvh")
twist_your_body_motion_df = twist_your_body_bvhp.get_motion_df()
twist_your_body_motion_df["label"] = 3

jump_bvhp = BVHparser("./data/bvh/jump.bvh")
jump_motion_df = jump_bvhp.get_motion_df()
jump_motion_df["label"] = 4

do_nothing_bvhp = BVHparser("./data/bvh/do_nothing.bvh")
do_nothing_motion_df = do_nothing_bvhp.get_motion_df()
do_nothing_motion_df["label"] = 5

skeleton = bending_exercise_bvhp.get_skeleton()

# 訓練データを作成
train_data_dict = convert2coordDict(motion_df, skeleton)
train_data = coord_dict2coords_array(train_data_dict)

# train_data を npy ファイルに保存
np.save("data/train_data.npy", train_data)

# ラベルを npy ファイルに保存
np.save("data/train_label.npy", motion_df["label"].values)

## テスト用データを生成するよ

In [5]:
all1_bvhp = BVHparser("./data/bvh/all1.bvh")
all1_motion_df = all1_bvhp.get_motion_df()
all1_motion_df["label"] = 0

all2_bvhp = BVHparser("./data/bvh/all2.bvh")
all2_motion_df = all2_bvhp.get_motion_df()
all2_motion_df["label"] = 0

# 全ての motion_df を結合
motion_df = pd.concat(
    [
        all1_motion_df,
        all2_motion_df,
    ]
)

# テストデータを作成
skeleton = all1_bvhp.get_skeleton()
test_data_dict = convert2coordDict(motion_df, skeleton)
test_data = coord_dict2coords_array(test_data_dict)

# test_data を npy ファイルに保存
np.save("data/test_data.npy", test_data)

# motion_df から label を取り出し、np.array に変換
np.save("data/test_label.npy", motion_df["label"].values)

# 骨格情報を生成するよ

In [6]:
print(train_data[0])

# train shape
print(train_data.shape)

# test shape
print(test_data.shape)

[[ 0.00000000e+00  9.32929000e+01  0.00000000e+00]
 [ 0.00000000e+00  9.83715700e+01 -1.15138000e+00]
 [ 0.00000000e+00  1.03988180e+02 -7.99500000e-02]
 [-1.02604000e-17  1.09977960e+02  3.50600000e-02]
 [-2.13061000e-17  1.16483790e+02 -4.08463000e-01]
 [-4.10356000e-17  1.24089270e+02 -1.91439300e+00]
 [-6.63663000e-17  1.33586640e+02 -2.82888500e+00]
 [-9.54064000e-17  1.44137940e+02 -1.25197500e+00]
 [-1.08325700e-16  1.48924380e+02 -5.36636000e-01]
 [-1.26441700e-16  1.53745470e+02 -1.14845000e-01]
 [-1.46537500e-16  1.58575880e+02  6.50217000e-01]
 [ 1.23299000e+00  1.36530630e+02  6.28176500e+00]
 [ 1.41876900e+01  1.39780130e+02  3.01304500e+00]
 [ 4.33957900e+01  1.39839938e+02  3.15008400e+00]
 [ 6.76297900e+01  1.39889560e+02  3.26378500e+00]
 [-1.23299000e+00  1.36530650e+02  6.28167500e+00]
 [-1.41876900e+01  1.39780150e+02  3.01295500e+00]
 [-4.33957900e+01  1.39839958e+02  3.14999400e+00]
 [-6.76297900e+01  1.39889580e+02  3.26369600e+00]
 [ 9.23673000e+00  8.90873400e+